# Predicción de Cantidades Vendidas por Región y Categoría con Prophet

Este notebook replica el flujo de predicción de cantidades vendidas por región y categoría, usando Prophet y considerando variables exógenas (`AVG_PRECIO_LISTA`, `AVG_DESC_PORCENTAJE`). Se generan pronósticos para dos escenarios, siguiendo la lógica de los modelos implementados en Snowflake.

## 1. Importar Librerías y Configuración

Importamos las librerías necesarias y configuramos el logging para trazabilidad.

In [14]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

## 2. Carga de Datos desde Snowflake

Cargamos los datos históricos y los dos escenarios de forecast desde las vistas generadas en Snowflake.

In [15]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='ringoquimicodev',  
            password='Ch3cooch2ch2ch3',  
            account='RSB72105', 
            warehouse='COMPUTE_WH', 
            database='BEBIDAS_PROJECT',
            schema='BEBIDAS_ANALYTICS'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise

def load_snowflake_views():
    conn = get_snowflake_connection()
    try:
        df_hist = pd.read_sql("SELECT * FROM VW_VENTAS_HISTORICO_M3", conn)
        df_fcst1 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_1", conn)
        df_fcst2 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_2", conn)
        logging.info(f"Histórico: {len(df_hist)} registros, Escenario 1: {len(df_fcst1)}, Escenario 2: {len(df_fcst2)}")
        return df_hist, df_fcst1, df_fcst2
    finally:
        conn.close()

df_hist, df_fcst1, df_fcst2 = load_snowflake_views()
print(f"Histórico: {df_hist.shape}, Escenario 1: {df_fcst1.shape}, Escenario 2: {df_fcst2.shape}")

2025-06-28 18:06:04,430 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.11, Platform: Windows-10-10.0.19045-SP0


2025-06-28 18:06:04,435 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-28 18:06:05,332 - INFO - Conexión a Snowflake exitosa
2025-06-28 18:06:11,424 - INFO - Histórico: 3984 registros, Escenario 1: 1680, Escenario 2: 1680


Histórico: (3984, 6), Escenario 1: (1680, 5), Escenario 2: (1680, 5)


In [16]:
producto = "Antioquia-Agua-Agua con Gas 500mL x 6uds"
df_hist = df_hist[df_hist['REGION_CATEGORIA_PRODUCTO'] == producto].copy()
df_fcst1 = df_fcst1[df_fcst1['REGION_CATEGORIA_PRODUCTO'] == producto].copy()
df_fcst2 = df_fcst2[df_fcst2['REGION_CATEGORIA_PRODUCTO'] == producto].copy()

## 3. Preparación de Datos para Prophet

Prophet requiere las columnas `ds` (fecha) y `y` (target). Además, agregamos las variables exógenas.

In [17]:
# Convertir MES a datetime y renombrar columnas
for df in [df_hist, df_fcst1, df_fcst2]:
    df['ds'] = pd.to_datetime(df['MES'])
df_hist['y'] = df_hist['M3_VENDIDOS']

# Seleccionar variables exógenas
exog_vars = ['AVG_PRECIO_LISTA', 'AVG_DESC_PORCENTAJE']

## 4. Entrenamiento del Modelo Prophet con Variables Exógenas

In [18]:
# Instanciar y agregar variables exógenas
m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
for var in exog_vars:
    m.add_regressor(var)

# Entrenar modelo
m.fit(df_hist[['ds', 'y'] + exog_vars])

2025-06-28 18:06:12,124 - DEBUG - cmd: where.exe tbb.dll
cwd: None


2025-06-28 18:06:12,694 - DEBUG - TBB already found in load path
2025-06-28 18:06:12,746 - INFO - n_changepoints greater than number of observations. Using 22.
2025-06-28 18:06:12,756 - DEBUG - input tempfile: C:\Users\joey_\AppData\Local\Temp\tmppdwpyuy0\bfm5nwnl.json
2025-06-28 18:06:12,766 - DEBUG - input tempfile: C:\Users\joey_\AppData\Local\Temp\tmppdwpyuy0\ifijtnac.json
2025-06-28 18:06:12,770 - DEBUG - idx 0
2025-06-28 18:06:12,772 - DEBUG - running CmdStan, num_threads: None
2025-06-28 18:06:12,774 - DEBUG - CmdStan args: ['C:\\Users\\joey_\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=12536', 'data', 'file=C:\\Users\\joey_\\AppData\\Local\\Temp\\tmppdwpyuy0\\bfm5nwnl.json', 'init=C:\\Users\\joey_\\AppData\\Local\\Temp\\tmppdwpyuy0\\ifijtnac.json', 'output', 'file=C:\\Users\\joey_\\AppData\\Local\\Temp\\tmppdwpyuy0\\prophet_model1jel4vxk\\prophet_model-20250628180612.csv', 'method=optimize', 'algorith

## 5. Predicción para Escenario 1 y Escenario 2

Se generan predicciones para ambos escenarios, usando las variables exógenas correspondientes.

In [19]:
# Preparar dataframes de forecast para Prophet
def prepare_future(df_fcst):
    return df_fcst[['ds'] + exog_vars].copy()

future1 = prepare_future(df_fcst1)
future2 = prepare_future(df_fcst2)

# Predicción escenario 1
forecast1 = m.predict(future1)
df_fcst1['CANTIDAD_VENDIDA_PRED'] = forecast1['yhat'].values

# Predicción escenario 2
forecast2 = m.predict(future2)
df_fcst2['CANTIDAD_VENDIDA_PRED'] = forecast2['yhat'].values

## 6. Métricas de Desempeño en el Histórico Reciente

Se evalúa el modelo en el histórico más reciente (por ejemplo, los últimos 7 meses).

In [20]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Evaluar en los últimos 7 meses del histórico
n_test = 7
test_hist = df_hist.tail(n_test)
future_test = test_hist[['ds'] + exog_vars]
forecast_test = m.predict(future_test)
y_true = test_hist['y'].values
y_pred = forecast_test['yhat'].values

print("Evaluación últimos 7 meses del histórico:")
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mean_absolute_error(y_true, y_pred):.2f}")
print(f"SMAPE: {smape(y_true, y_pred):.2f}%")
print(f"R2: {r2_score(y_true, y_pred):.2f}")

Evaluación últimos 7 meses del histórico:
RMSE: 0.09
MAE: 0.07
SMAPE: 4.60%
R2: 0.90


## 7. Visualización de Resultados

In [21]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

fig = go.Figure()

# 1. Histórico real completo
fig.add_trace(go.Scatter(
    x=df_hist['ds'],
    y=df_hist['y'],
    mode='lines',
    name='Histórico Real',
    line=dict(color='black')
))

# Línea de tendencia (regresión lineal sobre histórico)
X_trend = df_hist['ds'].map(lambda x: x.toordinal()).values.reshape(-1, 1)
y_trend = df_hist['y'].values
reg = LinearRegression().fit(X_trend, y_trend)
y_trend_pred = reg.predict(X_trend)
fig.add_trace(go.Scatter(
    x=df_hist['ds'],
    y=y_trend_pred,
    mode='lines',
    name='Tendencia (Histórico)',
    line=dict(color='gray', dash='dot')
))

# 2. Test real (últimos n_test meses)
fig.add_trace(go.Scatter(
    x=df_hist['ds'].tail(n_test),
    y=y_true,
    mode='lines',
    name='Histórico Test (Real)',
    line=dict(color='orange', dash='dash')
))

# 3. Predicción test (últimos n_test meses)
fig.add_trace(go.Scatter(
    x=df_hist['ds'].tail(n_test),
    y=y_pred,
    mode='lines',
    name='Histórico Test (Predicción)',
    line=dict(color='red', dash='dash')
))

# 4. Forecast Escenario 1
fig.add_trace(go.Scatter(
    x=df_fcst1['ds'],
    y=df_fcst1['CANTIDAD_VENDIDA_PRED'],
    mode='lines',
    name='Forecast Escenario 1',
    line=dict(color='blue', dash='dash')
))

# 5. Forecast Escenario 2
fig.add_trace(go.Scatter(
    x=df_fcst2['ds'],
    y=df_fcst2['CANTIDAD_VENDIDA_PRED'],
    mode='lines',
    name='Forecast Escenario 2',
    line=dict(color='green', dash='dash')
))

fig.update_layout(
    title=f'Pronóstico de Cantidades Vendidas por Región y Categoría (Prophet)\n{producto}',
    xaxis_title='Fecha',
    yaxis_title='Cantidad Vendida (m3)',
    legend_title='Serie',
    template='plotly_white',
    width=1200,
    height=600
)
fig.show()

## 8. Diferencia porcentual mes a mes

Se calcula y grafica la variación porcentual mes a mes para el histórico y ambos escenarios.

In [22]:
import plotly.subplots as sp
import plotly.graph_objects as go

def calc_diff(df, col, first_ref=None):
    diffs = df[col].pct_change() * 100
    if first_ref is not None and len(diffs) > 0:
        # Reemplaza el primer valor NaN por la diferencia con el último valor del histórico
        diffs.iloc[0] = 100 * (df[col].iloc[0] - first_ref) / ((abs(df[col].iloc[0]) + abs(first_ref)) / 2)
    return diffs

# Último valor real del histórico
last_hist = df_hist['y'].iloc[-1]

# Calcular diferencias
# Histórico: normal
# Escenarios: primer punto vs último histórico

df_hist['DIF_PCT'] = calc_diff(df_hist, 'y')
df_fcst1['DIF_PCT'] = calc_diff(df_fcst1, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)
df_fcst2['DIF_PCT'] = calc_diff(df_fcst2, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)

get_colors = lambda difs: [
    'green' if d > 0 else 'yellow' if d == 0 else 'red' for d in difs
]

fig = sp.make_subplots(
    rows=3, cols=1, 
    subplot_titles=['Histórico', 'Escenario 1', 'Escenario 2'],
    vertical_spacing=0.12
)

# 1. Histórico
fig.add_trace(
    go.Bar(
        x=df_hist['ds'],
        y=df_hist['DIF_PCT'],
        marker_color=get_colors(df_hist['DIF_PCT']),
        text=[f'{v:.1f}%' if not pd.isnull(v) else '' for v in df_hist['DIF_PCT']],
        textposition='outside',
        name='Histórico',
        width=20*24*60*60*1000
    ), row=1, col=1
)
# 2. Escenario 1
fig.add_trace(
    go.Bar(
        x=df_fcst1['ds'],
        y=df_fcst1['DIF_PCT'],
        marker_color=get_colors(df_fcst1['DIF_PCT']),
        text=[f'{v:.1f}%' if not pd.isnull(v) else '' for v in df_fcst1['DIF_PCT']],
        textposition='outside',
        name='Escenario 1',
        width=20*24*60*60*1000
    ), row=2, col=1
)
# 3. Escenario 2
fig.add_trace(
    go.Bar(
        x=df_fcst2['ds'],
        y=df_fcst2['DIF_PCT'],
        marker_color=get_colors(df_fcst2['DIF_PCT']),
        text=[f'{v:.1f}%' if not pd.isnull(v) else '' for v in df_fcst2['DIF_PCT']],
        textposition='outside',
        name='Escenario 2',
        width=20*24*60*60*1000
    ), row=3, col=1
)

fig.update_layout(
    title_text='Diferencia porcentual mes a mes',
    showlegend=False,
    height=1300,
    width=1200,
    bargap=0.25,
    font=dict(size=12),
    margin=dict(t=40, l=40, r=40, b=40)
)
for i in range(1, 4):
    fig.update_yaxes(title_text='% Dif.', row=i, col=1, zeroline=True, zerolinecolor='gray')
    fig.update_xaxes(tickformat='%b-%Y', row=i, col=1, tickangle=45)
fig.show()